# Arbitrage algorithm example

[example from here](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1096549)

In [1]:
import numpy as np
import pandas as pd

In [2]:
def compute_API(lambda_max, n):
    return np.abs(lambda_max-n)/(n-1)

In [3]:
def calculate_arbitrage(vecmax):
    B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
    C = np.divide(A, B)

    C_max = np.unravel_index(C.argmax(), C.shape)
    C_min = np.unravel_index(C.argmin(), C.shape)

    # Algorithm
    # BUY = division
    # SELL = multiply
    print('Arbitrage orders: ', end="")
    if C_max[0]==C_min[1] and C_max[1]==C_min[0]:
        # Direct arbitrage
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # and sell it for currency C_max[0] in location C_max[1]
        print('DIRECT')
        aer = np.real(1/C[C_min]*C[C_max]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]:6}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[0]]}({A[C_max[0],C_min[0]]}) in {locations[C_max[1]]:6}')

        operation = 1/A[C_min]*A[C_max]
        print(f'1/{A[C_min]:.4}*{A[C_max]:.4} = {operation:.5}')

    elif C_max[0]==C_min[0] or C_max[1]==C_min[1]:
        # Triangular arbitrage
        if C_max[0]==C_min[0]:
            # Arbitrage elements in the same row
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[1] in location C_max[1]
            # then buy currency C_max[1] in location C_max[1]
            print('TRIANGULAR ROW')
            aer = np.real(1/C[C_min]*C[C_min[0],C_max[1]]/C[C_min[1],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
            print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[1]]}({A[C_min[0],C_max[1]]}) in {locations[C_max[1]]}')
            print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[1]]/A[C_min[1],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
        else: # C_max[1]==C_min[1]
            # Arbitrage elements in the same col
            # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
            # then sell it for currency C_max[0] in location C_max[0]
            # then sell it for currency C_min[1] in location C_max[1]
            print('TRIANGULAR COLUMN')
            aer = np.real(1/C[C_min]*C[C_min[0], C_max[0]]*C[C_max[0],C_max[1]]-1)

            print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
            print(f'BUY {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
            print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
            print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[1]]}({A[C_max[0],C_min[1]]}) in {locations[C_max[1]]}')
            operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]
            print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4} = {operation:.5}')
    else:
        # Cuadrangular arbitrage
        # Arbitrage that involves four currencies and four locations
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[0] in location C_max[0]
        # then sell it for currency C_max[1] in location C_max[1]
        # then buy currency C_min[1] in location C_max[1]
        print('CUADRANGULAR')
        aer = np.real(1/C[C_min]*C[C_min[0],C_max[0]]*C[C_max]/C[C_min[1],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_max[1]]}({A[C_max[0],C_max[1]]}) in {locations[C_max[1]]}')
        print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]/A[C_min[1],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')

In [4]:
currencies = ['USDT', 'EUR', 'POUND', 'YEN', 'HKDOLLAR', 'SINGDOLLAR']
locations = ['NY', 'FRANK', 'LOND', 'TOKYO', 'HK', 'SING']

In [5]:
A = np.array([[1, 1.1038, 0.6888, 1.19, 7.8, 1.8235],
               [0.905, 1, 0.6241, 1.0955, 7.0801, 1.6403],
               [1.4501, 1.6026, 1, 1.7705, 11.32, 2.6506],
               [0.8893, 0.9099, 0.5677, 1, 6.43, 1.4957],
               [0.1282, 0.1414, 0.0883, 0.1555, 1, 0.2328],
               [0.5521, 0.6098, 0.3773, 0.6681, 4.3100, 1]])
A

array([[ 1.    ,  1.1038,  0.6888,  1.19  ,  7.8   ,  1.8235],
       [ 0.905 ,  1.    ,  0.6241,  1.0955,  7.0801,  1.6403],
       [ 1.4501,  1.6026,  1.    ,  1.7705, 11.32  ,  2.6506],
       [ 0.8893,  0.9099,  0.5677,  1.    ,  6.43  ,  1.4957],
       [ 0.1282,  0.1414,  0.0883,  0.1555,  1.    ,  0.2328],
       [ 0.5521,  0.6098,  0.3773,  0.6681,  4.31  ,  1.    ]])

In [6]:
%%time

eigvals, eigvects = np.linalg.eig(A)
idxmax = np.argmax(eigvals)
valmax = eigvals[idxmax]
valapi = compute_API(valmax, len(A))
vecmax = eigvects[:,idxmax]

if valapi>0:
    print(f'Arbitrage oportunity detected. API={valapi:.4}')
    calculate_arbitrage(vecmax)
else:
    print(f'Arbitrage oportunity no detected. API={valapi:.4}')

Arbitrage oportunity detected. API=0.002311
Arbitrage orders: TRIANGULAR ROW
EUR -> YEN -> USDT -> EUR : AER = 7.996%
BUY  YEN/EUR(0.9099) in FRANK
SELL YEN/USDT(0.8893) in NY
BUY  EUR/USDT(0.905) in NY
1/0.9099*0.8893/0.905 = 1.08
CPU times: user 668 µs, sys: 0 ns, total: 668 µs
Wall time: 551 µs


---

In [11]:
B = np.fromfunction(lambda i, j: vecmax[i] / vecmax[j], (len(A),len(A)), dtype=int)
C = np.divide(A, B)

C_max = np.unravel_index(C.argmax(), C.shape)
C_min = np.unravel_index(C.argmin(), C.shape)

# Algorithm
# BUY = division
# SELL = multiply
print('Arbitrage orders: ', end="")
if C_max[0]==C_min[1] and C_max[1]==C_min[0]:
    # Direct arbitrage
    # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
    # and sell it for currency C_max[0] in location C_max[1]
    print('DIRECT')
    aer = np.real(1/C[C_min]*C[C_max]-1)

    print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
    print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]:6}')
    print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[0]]}({A[C_max[0],C_min[0]]}) in {locations[C_max[1]]:6}')

    operation = 1/A[C_min]*A[C_max]
    print(f'1/{A[C_min]:.4}*{A[C_max]:.4} = {operation:.5}')

elif C_max[0]==C_min[0] or C_max[1]==C_min[1]:
    # Triangular arbitrage
    if C_max[0]==C_min[0]:
        # Arbitrage elements in the same row
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[1] in location C_max[1]
        # then buy currency C_max[1] in location C_max[1]
        print('TRIANGULAR ROW')
        aer = np.real(1/C[C_min]*C[C_min[0],C_max[1]]/C[C_min[1],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[1]]}({A[C_min[0],C_max[1]]}) in {locations[C_max[1]]}')
        print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[1]]/A[C_min[1],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')
    else: # C_max[1]==C_min[1]
        # Arbitrage elements in the same col
        # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
        # then sell it for currency C_max[0] in location C_max[0]
        # then sell it for currency C_min[1] in location C_max[1]
        print('TRIANGULAR COLUMN')
        aer = np.real(1/C[C_min]*C[C_min[0], C_max[0]]*C[C_max[0],C_max[1]]-1)

        print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
        print(f'BUY {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
        print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
        print(f'SELL {currencies[C_max[0]]}/{currencies[C_min[1]]}({A[C_max[0],C_min[1]]}) in {locations[C_max[1]]}')
        operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]
        print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4} = {operation:.5}')
else:
    # Cuadrangular arbitrage
    # Arbitrage that involves four currencies and four locations
    # Use currency C_min[1] and buy currency C_min[0] in location C_min[1]
    # then sell it for currency C_max[0] in location C_max[0]
    # then sell it for currency C_max[1] in location C_max[1]
    # then buy currency C_min[1] in location C_max[1]
    print('CUADRANGULAR')
    aer = np.real(1/C[C_min]*C[C_min[0],C_max[0]]*C[C_max]/C[C_min[1],C_max[1]]-1)

    print(f'{currencies[C_min[1]]} -> {currencies[C_min[0]]} -> {currencies[C_max[0]]} -> {currencies[C_max[1]]} -> {currencies[C_min[1]]} : AER = {aer:.3%}')
    print(f'BUY  {currencies[C_min[0]]}/{currencies[C_min[1]]}({A[C_min]}) in {locations[C_min[1]]}')
    print(f'SELL {currencies[C_min[0]]}/{currencies[C_max[0]]}({A[C_min[0],C_max[0]]}) in {locations[C_max[0]]}')
    print(f'SELL {currencies[C_max[0]]}/{currencies[C_max[1]]}({A[C_max[0],C_max[1]]}) in {locations[C_max[1]]}')
    print(f'BUY  {currencies[C_min[1]]}/{currencies[C_max[1]]}({A[C_min[1],C_max[1]]}) in {locations[C_max[1]]}')
    operation = 1/A[C_min]*A[C_min[0],C_max[0]]*A[C_max[0],C_max[1]]/A[C_min[1],C_max[1]]
    print(f'1/{A[C_min]:.4}*{A[C_min[0],C_max[0]]:.4}*{A[C_max[0],C_max[1]]:.4}/{A[C_min[1],C_max[1]]:.4} = {operation:.5}')

Arbitrage orders: TRIANGULAR ROW
EUR -> YEN -> USDT -> EUR : AER = 7.996%
BUY  YEN/EUR(0.9099) in FRANK
SELL YEN/USDT(0.8893) in NY
BUY  EUR/USDT(0.905) in NY
1/0.9099*0.8893/0.905 = 1.08


In [12]:
x = 1/1.095
y=x*0.9099
print(f'{y-1:.4%}')

-16.9041%


In [13]:
eigvals

array([ 6.01155494e+00+0.j        , -5.44904137e-03+0.03210245j,
       -5.44904137e-03-0.03210245j, -2.75215004e-04+0.00606689j,
       -2.75215004e-04-0.00606689j, -1.06431030e-04+0.j        ])

In [23]:
np.real(eigvects).T

array([[-0.44852453, -0.40701659, -0.65381971, -0.37545331, -0.05763717,
        -0.24778175],
       [-0.11670674, -0.1726539 , -0.35662142,  0.72987261, -0.02473111,
        -0.07454008],
       [-0.11670674, -0.1726539 , -0.35662142,  0.72987261, -0.02473111,
        -0.07454008],
       [ 0.02678808,  0.32050237, -0.8342348 , -0.0063796 , -0.00818839,
         0.14561592],
       [ 0.02678808,  0.32050237, -0.8342348 , -0.0063796 , -0.00818839,
         0.14561592],
       [ 0.01371163, -0.94628762, -0.26515278,  0.00466774,  0.12264773,
         0.13777326]])

In [19]:
currencies

['USDT', 'EUR', 'POUND', 'YEN', 'HKDOLLAR', 'SINGDOLLAR']

In [22]:
pd.Series(np.real(vecmax), index=currencies)

USDT         -0.448525
EUR          -0.407017
POUND        -0.653820
YEN          -0.375453
HKDOLLAR     -0.057637
SINGDOLLAR   -0.247782
dtype: float64